## Data preparation pipeline
- Load file to prepare
- Clean the file
- Create an index
- Concatenate



## Functions

In [1]:
from bs4 import BeautifulSoup
import os
import unicodedata
import json
from pprint import pprint as pp
import re

In [2]:
# open the file Jerusalem_Talmud_complete.json
with open('Babylon_Talmud_complete.json', 'r') as f:
    data = json.load(f)

for book in data:
    print(len(book['sectionNames']))
    print(book['sectionNames'])

2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']
2
['Daf', 'Line']


In [3]:
def identify_book_structure(file):
    '''
    Identifies the structure of the book
    Returns:
    - 'book_simple' if json contains a single book with a simple structure (e.g. chapter -> verse (-> subverse if needed))
    - 'corpus_simple' if json contains a corpus of books (like Tanakh or Talmud) with a simple structure
    - 'book_complex' if json contains a single book with complex structure (with nodes)
    - 'corpus_complex' if json contains a corpus of books with complex structure (with nodes)
    More about the Sefaria text structures:
    https://developers.sefaria.org/docs/the-index-schema
    '''
    # Open the json file and load the content
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Check if data is a dictionary
    if isinstance(data, dict):
        if 'text' in data:
            if isinstance(data['text'], list):
                # Check if the first element is a dictionary
                if len(data['text']) > 0 and isinstance(data['text'][0], dict):
                    return 'book_simple'
                return 'book_simple'  # For simple structures
            elif isinstance(data['text'], dict):
                return 'book_complex'

    # Check if data is a list of dictionaries
    elif isinstance(data, list):
        if len(data) > 0 and isinstance(data[0], dict):
            if 'text' in data[0]:
                if isinstance(data[0]['text'], list):
                    # Check if the first element in 'text' is a dictionary
                    if len(data[0]['text']) > 0 and isinstance(data[0]['text'][0], dict):
                        return 'corpus_complex'
                    return 'corpus_simple'
                elif isinstance(data[0]['text'], dict):
                    return 'corpus_complex'
    
    # Default return for any other structure
    return 'unknown_structure'

In [4]:
file = 'Babylon_Talmud_complete.json'
identify_book_structure(file)

'corpus_simple'

In [5]:
def browse_simple_book_structure_and_show_unique_chars(filename):
    '''
    This function reads a simple structured file containing a single book and returns a set of unique characters in the file.
    The aim is to quickly identify characters that should be removed from the text.
    '''

    with open(filename, "r") as f:
        data = json.load(f)

    character_set = set()

    for chapter in data["text"]:
        for verse in chapter:
            if isinstance(verse, str):
                character_set.update(verse)
            elif isinstance(verse, list):
                for sub_verse in verse:
                    character_set.update(sub_verse)

    return character_set

In [6]:
def browse_complex_book_structure_and_show_unique_chars(filename):

    '''
    This function reads a complex structured file containing a single book and returns a set of unique characters in the file.
    The aim is to quickly indentify characters that should be removed from the text.
    '''

    def browse_and_show_unique_chars(nodes, level=0, character_set=None):
        '''
        recursively browse the nodes and return a set of unique characters in the file.
        '''

        if character_set is None:
            character_set = set()

        for value in nodes.values():
            # If the node value is a list, add the unique characters of each list item to the set
            if isinstance(value, list):
                for elem in value:
                    # If the element is a list, iterate through its items and add their unique characters to the set
                    if isinstance(elem, list):
                        for sub_elem in elem:
                            if isinstance(sub_elem, list):
                                for sub_sub_elem in sub_elem:
                                    character_set.update(sub_sub_elem)                         
                            else:
                                character_set.update(sub_elem)
                    else:
                        character_set.update(elem)

            # If the node value is a dictionary, recursively call the browse_nodes function on the dictionary.
            elif isinstance(value, dict):
                browse_and_show_unique_chars(value, level + 1, character_set)

        return character_set

    with open(filename, "r") as f:
        data = json.load(f)

    nodes = data['text']

    return browse_and_show_unique_chars(nodes, level=0, character_set=None)


In [7]:
def browse_simple_corpus_structure_and_show_unique_chars(filename):
    '''
    This function reads a simple structured file containing a corpus of books and returns a set of unique characters in the file.
    The aim is to quickly identify characters that should be removed from the text.
    '''

    with open(filename, "r") as f:
        data = json.load(f)

    character_set = set()
    for book in data:
        for chapter in book["text"]:
            for verse in chapter:
                if isinstance(verse, str):
                    character_set.update(verse)
                elif isinstance(verse, list):
                    for sub_verse in verse:
                        character_set.update(sub_verse)
    return character_set

In [8]:
#test
file = 'Babylon_Talmud_complete.json'
browse_simple_corpus_structure_and_show_unique_chars(file)


{'\n',
 ' ',
 '!',
 '"',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '>',
 '?',
 '[',
 '\\',
 ']',
 'b',
 'g',
 'i',
 'n',
 'o',
 'r',
 's',
 't',
 '{',
 '}',
 '\xa0',
 'ְ',
 'ֱ',
 'ֲ',
 'ֳ',
 'ִ',
 'ֵ',
 'ֶ',
 'ַ',
 'ָ',
 'ֹ',
 'ֻ',
 'ּ',
 'ֽ',
 'ֿ',
 'ׁ',
 'ׂ',
 'ׇ',
 'א',
 'ב',
 'ג',
 'ד',
 'ה',
 'ו',
 'ז',
 'ח',
 'ט',
 'י',
 'ך',
 'כ',
 'ל',
 'ם',
 'מ',
 'ן',
 'נ',
 'ס',
 'ע',
 'ף',
 'פ',
 'ץ',
 'צ',
 'ק',
 'ר',
 'ש',
 'ת',
 '׳',
 '״',
 '\u200e',
 '–',
 '—',
 '…',
 'הּ'}

In [9]:
def browse_file_and_show_unique_chars(file):
    '''
    Show unique characters in the file, depending on the structure of the file (simple or complex, book or corpus)
    '''
    if identify_book_structure(file) == 'book_simple':
        return browse_simple_book_structure_and_show_unique_chars(file)
    elif identify_book_structure(file) == 'book_complex':
        return browse_complex_book_structure_and_show_unique_chars(file)
    elif identify_book_structure(file) == 'corpus_simple':
        return browse_simple_corpus_structure_and_show_unique_chars(file)
    else:
        return 'Unknown structure, cannot browse the file.'

In [10]:
file = 'cleaned/Jerusalem_Talmud_complete_clean.json'
browse_file_and_show_unique_chars(file)

{' ',
 '"',
 '&',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '[',
 ']',
 '|',
 '·',
 '͏',
 '־',
 '׀',
 '׃',
 'א',
 'ב',
 'ג',
 'ד',
 'ה',
 'ו',
 'ז',
 'ח',
 'ט',
 'י',
 'ך',
 'כ',
 'ל',
 'ם',
 'מ',
 'ן',
 'נ',
 'ס',
 'ע',
 'ף',
 'פ',
 'ץ',
 'צ',
 'ק',
 'ר',
 'ש',
 'ת',
 'װ',
 '׳',
 '״',
 '‘'}

In [10]:
def browse_simple_corpus_structure_and_show_specific_char(filename, target_character):
    '''
    This function reads a simple structured file containing a corpus of books and returns lines containing a specified character.
    '''

    with open(filename, "r") as f:
        data = json.load(f)

    lines_with_target_char = []
    for book in data:
        for chapter in book["text"]:
            for verse in chapter:
                if isinstance(verse, str):
                    if target_character in verse:
                        lines_with_target_char.append(verse)
                elif isinstance(verse, list):
                    for sub_verse in verse:
                        if target_character in sub_verse:
                            lines_with_target_char.append(sub_verse)
    
    return lines_with_target_char

In [15]:
filename = 'cleaned/Jerusalem_Talmud_complete_clean.json'
target_character = '&'
browse_simple_corpus_structure_and_show_specific_char(filename, target_character)

['הכל מודין שאין ממון אצל מיתה דכתיב ומכה (נפש) בהמה ישלמנה ומכה אדם יומת. מה מכה בהמה לא חלקתה בה בין שוגג בין מזיד &;לפטור&; [לחייבו] < -="   " -=""> ממון אף מכה אדם לא תחלוק בו בין שוגג בין מזיד לפטור. במה פליגין. בממון אצל מכות. רבי יוחנן אמר אין ממון אצל מיתה ויש ממון אצל מכות. ורבי שמעון בן לקיש אמר כשם שאין ממון אצל מיתה כך אין ממון אצל מכות.',
 'רבי &;אחא&; בשם רב. אסור לדוש על גבי זמורה בשבת. היא שורשי אילן היא קולסי אכרוב. בגבוהין שלשה. אבל אם אינן גבוהין שלשה &;כ&;ארץ הם.',
 '[שני אחים הוון בכפר חריבה. והוון רומים אזלין עליהון ומקטלין לון. ואמרין. כל־סמא דמילתא ניתי כלילא על רישיהון. אמרין. מבדקינן אוף חד זמן. מנפקין פגע ביה חד סב. אמר לון. ברייכון סעודינכון. אמר. לא יסעוד ולא [יסמוך] &;יכסוף&; הלא־אתה אלקים זנחתנו.]',
 'על־כל המתים הוא אסור לילך בסעודה עד שלשים יום. על אביו ועל אמו עד שנים עשר חודש. אם היתה חבורת מצוה או קידוש החודש &;מותר&;.',
 '&;הרי&; שהיה מחליף בגדים כל־שבעה חייב לקרוע את כולן. רבי חייה רבה ורבי חמא אבוי דרבי הושעיה תריהון אמרין. כולהן אסורין באיחוי. בר

In [11]:
def create_index_from_simple_book(filename):
    '''
    Browses simple structures files and create index entries.
    With the index, we can then quickly identify the reference of a text chunk,
    knowing the start and end character position of the text chunk in the book.
    '''
    # Load the file
    with open(filename, "r") as f:
        data = json.load(f)

    # Create a list to store index entries
    index = {}
    # add the title of the book to the index
    index["title"] = data["title"]
    index['categories'] = data['categories']
    index['sectionNames'] = data['sectionNames']
    index['lines'] = []

    # Generate a unique identifier for each of the smallest text unit,
    # E.g. verses.
    def generate_uid(title, number):
        return f"{title.replace(' ', '_')}_{number}"

    # Counts the chars, and fills the index with start and end char position
    # Hebrew diacritics are removed from the count.
    def browse_text_and_count_verses_chars(text):
        def remove_hebrew_diacritics(text):
            normalized_text = unicodedata.normalize('NFKD', text)
            return ''.join(c for c in normalized_text if not unicodedata.combining(c))

        # Variable to track unique identification number
        uid = 1
        # Initialize character counter
        verse_start_char = 1
        # Intialize chapter counter
        chapter_number = 1

        for chapter in data['text']:
            # Initialize verse counter
            verse_number = 1

            for verse in chapter:
                if isinstance(verse, str):
                    verse = remove_hebrew_diacritics(verse)
                    verse_end_char = verse_start_char + len(verse) - 1

                    # Add verse to index with UID
                    index['lines'].append({
                        "uid": generate_uid(index['title'], uid),  # UID unique
                        "chapter_number": chapter_number,
                        "verse_number": verse_number,
                        "start_char": verse_start_char,
                        "end_char": verse_end_char,
                        "length": len(verse),
                        "text": verse
                    })

                    # Increment UID for next verse
                    uid += 1

                    # Increment verse number for next verse
                    verse_number += 1

                    # Update character counter for next verse
                    verse_start_char = verse_end_char + 1
                elif isinstance(verse, list):

                    # Initialize subverse counter
                    sub_verse_number = 1

                    for sub_verse in verse:
                        sub_verse = remove_hebrew_diacritics(sub_verse)
                        sub_verse_end_char = verse_start_char + len(sub_verse) - 1

                        # Add verse to index with UID
                        index['lines'].append({
                            "uid": generate_uid(index['title'], uid),  # UID unique
                            "chapter_number": chapter_number,
                            "verse_number": verse_number,
                            "sub_verse_number": sub_verse_number,
                            "start_char": verse_start_char,
                            "end_char": sub_verse_end_char,
                            "length": len(sub_verse),
                            "text": sub_verse
                        })

                        # Increment UID for next verse
                        uid += 1

                        # Increment verse number for next verse
                        sub_verse_number += 1

                        # Update character counter for next verse
                        verse_start_char = sub_verse_end_char + 1
                    verse_number += 1
            chapter_number += 1
    
    # Load the nodes
    text = data['text']
    browse_text_and_count_verses_chars(text)

    # Create a new directory named "indexes" if it doesn't exist
    index_dir = "indexes"
    if not os.path.exists(index_dir):
        os.makedirs(index_dir)

    # Save the index to a file, in the "indexes" directory
    title = index["title"]
    index_filename = os.path.join(index_dir, "index_" + title.replace(' ', '_') + ".json")
    with open(index_filename, "w") as f:
        json.dump(index, f, indent=2, ensure_ascii=False)


In [12]:
def create_index_from_complex_book(filename):
    '''
    Browses complex structured files and create index entries.
    With the index, we can then quickly identify the reference of a text chunk,
    knowing the start and end character position of the text chunk in the book.
    '''

    # Load the file
    with open(filename, "r") as f:
        data = json.load(f)

    # Create a list to store index entries
    index = {}
    # add the title of the book to the index
    index["title"] = data["title"]
    index['categories'] = data['categories']
    index['lines'] = []

    def generate_uid(title, number):
        return f"{title.replace(' ', '_')}_{number}"

    def browse_nodes_and_count_verses_chars(nodes, parent_keys=[], level=0):
        def remove_hebrew_diacritics(text):
            normalized_text = unicodedata.normalize('NFKD', text)
            return ''.join(c for c in normalized_text if not unicodedata.combining(c))

        # Variable to track unique identification number
        uid = 1

        # Initialize character counter
        verse_start_char = 1

        item_lvl_1_number, item_lvl_2_number, item_lvl_3_number = 0, 0, 0


        for key, value in nodes.items():
            # Add the current key to the list of parent keys
            current_keys = parent_keys + [key]

            if isinstance(value, list):
                # If the node value is a list, check if it contains strings or lists

                # Initialize verse counter
                item_lvl_1_number = 1

                for item_lvl_1 in value:
                # 1st lvl: If the item is a string, treat it as a single verse
                    if isinstance(item_lvl_1, str):
                        # remove hebrew vowels
                        item_lvl_1 = remove_hebrew_diacritics(item_lvl_1)
                        verse_end_char = verse_start_char + len(item_lvl_1) - 1

                        # Add verse to index with UID
                        index['lines'].append({
                            "uid": generate_uid(index['title'], uid),  # UID unique
                            "parent_titles": current_keys,
                            "item_lvl_1_number": item_lvl_1_number,
                            "start_char": verse_start_char,
                            "end_char": verse_end_char,
                            "length": len(item_lvl_1),
                            "text": item_lvl_1
                        })

                        # Increment UID for next verse
                        uid += 1


                        # Update character counter for next verse
                        verse_start_char = verse_end_char + 1


                    # 1st lvl: if the item in value is a list, treat it as a list
                    if isinstance(item_lvl_1, list):
                        # Initialize verse counter
                        item_lvl_2_number = 1
                        for item_lvl_2 in item_lvl_1:
                            # 2nd lvl:
                            #  if item is a string, treat it as a single verse
                            if isinstance(item_lvl_2, str):

                                # remove hebrew vowels
                                item_lvl_2 = remove_hebrew_diacritics(item_lvl_2)
                                verse_end_char = verse_start_char + len(item_lvl_2) - 1

                                # Add verse to index with UID
                                index['lines'].append({
                                    "uid": generate_uid(index['title'], uid),  # UID unique
                                    "parent_titles": current_keys,
                                    "item_lvl_1_number": item_lvl_1_number,
                                    "item_lvl_2_number": item_lvl_2_number,
                                    "start_char": verse_start_char,
                                    "end_char": verse_end_char,
                                    "length": len(item_lvl_2),
                                    "text": item_lvl_2
                                })

                                # Increment UID for next verse
                                uid += 1

                                # Increment verse number for next verse
                                item_lvl_2_number += 1

                                # Update character counter for next verse
                                verse_start_char = verse_end_char + 1

                            # 2nd lvl: if the item is a list
                            elif isinstance(item_lvl_2, list):
                                # Initialize subverse counter
                                item_lvl_3_number = 1
                                for item_lvl_3 in item_lvl_2:
                                
                                    # 3rd lvl: if the sub_verse is a string
                                    if isinstance(item_lvl_3, str):
                                        item_lvl_3 = remove_hebrew_diacritics(item_lvl_3)
                                        verse_end_char = verse_start_char + len(item_lvl_3) - 1

                                        # Add verse to index with UID
                                        index['lines'].append({
                                        "uid": generate_uid(index['title'], uid),  # UID unique
                                        "parent_titles": current_keys,
                                        "item_lvl_1_number": item_lvl_1_number,
                                        "item_lvl_2_number": item_lvl_2_number,
                                        "item_lvl_3_number": item_lvl_3_number,
                                        "start_char": verse_start_char,
                                        "end_char": verse_end_char,
                                        "length": len(item_lvl_3),
                                        "text": item_lvl_3
                                        })

                                        # Increment UID for next verse
                                        uid += 1

                                        # Increment verse number for next verse
                                        item_lvl_3_number += 1

                                        # Update character counter for next verse
                                        verse_start_char = verse_end_char + 1

                                    # 3rd lvl: if the sub_verse is a list
                                    if isinstance(item_lvl_3, list):
                                        print("nested verses detected and not handled !")

                                item_lvl_2_number += 1
                        # Increment verse number for next verse
                        item_lvl_1_number += 1


            # If the node value is a dictionary, recursively call the browse_nodes_and_count_verses_chars function on the dictionary.
            elif isinstance(value, dict):
                browse_nodes_and_count_verses_chars(value, parent_keys=current_keys, level=level + 1)

    # Load the nodes
    nodes = data['text']
    browse_nodes_and_count_verses_chars(nodes, level=0)

    # Create a new directory named "indexes" if it doesn't exist
    index_dir = "indexes"
    if not os.path.exists(index_dir):
        os.makedirs(index_dir)

    # Save the index to a file, in the "indexes" directory
    title = index["title"]
    index_filename = os.path.join(index_dir, "index_" + title.replace(' ', '_') + ".json")
    with open(index_filename, "w") as f:
        json.dump(index, f, indent=2, ensure_ascii=False)


In [13]:
def get_section_names_in_simple_corpus(data):
    '''
    This function returns the number of sections and the section names in a simple structured corpus.
    If the section number is not the same for all books:
    - print("All books do not have the same number of sections.")
    - print the max number of sections and the section names for the book with the max number of sections.
    '''    

    # Initialize variables to track section information
    section_counts = []
    section_names = []

    for book in data:
        # Get the number of sections and section names for each book
        number_of_sections = len(book['sectionNames'])
        section_counts.append(number_of_sections)
        section_names.append(book['sectionNames'])

    # Check if the number of sections is consistent across all books
    if len(set(section_counts)) == 1:
        # If the number of sections is the same for all books
        num_sections = section_counts[0]  # Number of sections
        names = section_names[0]  # Section names
        return num_sections, names
    else:
        # If the number of sections is different for some books
        print("All books do not have the same number of sections.")
        max_sections = max(section_counts)
        max_index = section_counts.index(max_sections)
        max_names = section_names[max_index]
        print(f"Maximum number of sections: {max_sections}")
        print("Section names for the book with the maximum number of sections:")
        for name in max_names:
            print(name)
        return (max_sections, max_names)

In [14]:
file = 'Babylon_Talmud_complete.json'
with open(file, 'r') as f:
    data = json.load(f)

get_section_names_in_simple_corpus(data)

(2, ['Daf', 'Line'])

In [16]:
import json
import os

def create_index_from_simple_corpus(filename):
    '''
    Browses a simple structured corpus and create index entries.
    With the index, we can then quickly identify the reference of a text chunk,
    knowing the start and end character position of the text chunk in the book.
    '''
    # Load the file
    if not os.path.exists(filename):
        print(f"File {filename} not found.")
        return

    with open(filename, "r") as f:
        data = json.load(f)

    # Assuming the function get_section_names_in_simple_corpus exists and is defined elsewhere
    nb_sections, section_names = get_section_names_in_simple_corpus(data)

    # Create a list to store index entries
    index = {}

    # Add the title of the corpus to the index
    index['title'] = os.path.basename(filename).split('.')[0]
    index['categories'] = data[0]['categories'][:2]
    index['sectionNames'] = section_names
    index['lines'] = []

    # Generate a unique identifier for each of the smallest text units
    def generate_uid(title, number):
        return f"{title.replace(' ', '_')}_{number}"

    # Variable to track unique identification number
    uid = 1

    for book in data:
        # Initialize character counter
        verse_start_char = 1
        # Initialize chapter counter
        chapter_number = 1

        for chapter in book['text']:
            # Initialize verse counter
            verse_number = 1

            for verse in chapter:
                if isinstance(verse, str):
                    verse_end_char = verse_start_char + len(verse) - 1

                    # Add verse to index with UID
                    index['lines'].append({
                        "uid": generate_uid(index['title'], uid),  # UID unique
                        "book_title": book["title"],
                        "chapter_number": chapter_number,
                        "verse_number": verse_number,
                        "start_char": verse_start_char,
                        "end_char": verse_end_char,
                        "length": len(verse),
                        "text": verse
                    })

                    # Increment UID for next verse
                    uid += 1

                    # Increment verse number for next verse
                    verse_number += 1

                    # Update character counter for next verse
                    verse_start_char = verse_end_char + 1
                elif isinstance(verse, list):
                    # Initialize subverse counter
                    sub_verse_number = 1

                    for sub_verse in verse:
                        sub_verse_end_char = verse_start_char + len(sub_verse) - 1

                        # Add verse to index with UID
                        index['lines'].append({
                            "uid": generate_uid(index['title'], uid),  # UID unique
                            "book_title": book["title"],
                            "chapter_number": chapter_number,
                            "verse_number": verse_number,
                            "sub_verse_number": sub_verse_number,
                            "start_char": verse_start_char,
                            "end_char": sub_verse_end_char,
                            "length": len(sub_verse),
                            "text": sub_verse
                        })

                        # Increment UID for next verse
                        uid += 1

                        # Increment subverse number for next subverse
                        sub_verse_number += 1

                        # Update character counter for next subverse
                        verse_start_char = sub_verse_end_char + 1
                    verse_number += 1
            chapter_number += 1    

    # Create a new directory named "indexes" if it doesn't exist
    index_dir = "indexes"
    if not os.path.exists(index_dir):
        os.makedirs(index_dir)

    # Save the index to a file, in the "indexes" directory
    title = index["title"]
    index_filename = os.path.join(index_dir, "index_" + title.replace(' ', '_') + ".json")
    with open(index_filename, "w") as f:
        json.dump(index, f, indent=2, ensure_ascii=False)

# File to process
file = 'cleaned/Babylon_Talmud_complete_clean.json'

# Create the index
create_index_from_simple_corpus(file)


File cleaned/Babylon_Talmud_complete_clean.json not found.


In [17]:
def browse_and_clean(filename):

    '''
    Remove unwanted characters from the text (e.g. numbers, special unicodes, html tags, etc.)
    Handles both simple and complex structured files.
    '''
    def remove_latin_characters(text):
        return re.sub(r'[a-zA-Z]', '', text)

    def remove_hebrew_diacritics(text):
        normalized_text = unicodedata.normalize('NFKD', text)
        return ''.join(c for c in normalized_text if not unicodedata.combining(c))
    
    def remove_angle_brackets(text):
        return re.sub(r'\<.*?\>', '', text)
    
    def remove_latin_punctuation(text):
        chars_to_remove = {'&', '?', '*', '='}
        for char in chars_to_remove:
            text = text.replace(char, '')
        return text

    def remove_special_unicodes(text):
        chars_to_remove = {'\n', '\u2003', '\u200d', '\u200e', '\u202c','\u2009', '\xa0' ,'\xad', '\u200c', '\u200f'}
        for char in chars_to_remove:
            text = text.replace(char, '')
        return text

    def remove_numbers(text):
        return ''.join(char for char in text if not char.isdigit())
    
    def clean_html(text):
        if text.strip():  # Checks if text is not empty after removing spaces
            soup = BeautifulSoup(text, 'html.parser')
            
            # Specifically deletes <b> and </b> tags
            for bold_tag in soup.find_all(['b', 'strong']):
                bold_tag.unwrap()
            
            return soup.get_text()
        else:
            return text  # Returns text unchanged if empty

    # we try to catch and display the MarkupResemblesLocatorWarning warning
    # import warnings

    # def clean_html(text):
    #     with warnings.catch_warnings(record=True) as w:
    #         warnings.simplefilter("always")  # Capture tous les avertissements
    #         if text.strip():
    #             text_to_parse = text  # Stocker le texte avant de l'analyser
    #             soup = BeautifulSoup(text, 'html.parser')
    #             text_without_html = soup.get_text()
    #             # Parcourir tous les avertissements capturés
    #             for warning in w:
    #                 if "MarkupResemblesLocatorWarning" in str(warning.message):
    #                     print("MarkupResemblesLocatorWarning captured while parsing the following text:")
    #                     print(text_to_parse)
    #             return text
    #         else:
    #             return text
  
    
    def remove_braces(text):
        return re.sub(r'\{.*?\}', '', text)
   

    def clean_text(text):
        
        text = remove_latin_characters(text)
        text = remove_latin_punctuation(text)
        text = remove_hebrew_diacritics(text)
        text = clean_html(text)
        text = remove_special_unicodes(text)
        text = remove_numbers(text)
        text = remove_braces(text)
        text = remove_angle_brackets(text)

        return text

    with open(filename, "r") as f:
        data = json.load(f)

    text_structure = identify_book_structure(filename)

    if text_structure == 'corpus_simple':
        for book in data:
            for chapter in book["text"]:
                for i, verse in enumerate(chapter):
                    if isinstance(verse, str):
                        chapter[i] = clean_text(verse)
                    elif isinstance(verse, list):
                        for j, sub_verse in enumerate(verse):
                            verse[j] = clean_text(sub_verse)

    elif isinstance(data['text'], list):

        # Browse and clean verses directly in the data structure
        for chapter in data["text"]:
            if isinstance(chapter, str):
                chapter = clean_text(chapter)
            else:
                for i, verse in enumerate(chapter):
                    if isinstance(verse, str):
                        chapter[i] = clean_text(verse)
                    elif isinstance(verse, list):
                        for j, sub_verse in enumerate(verse):
                            verse[j] = clean_text(sub_verse)        
    
    
    elif isinstance(data['text'], dict):
        
        def browse_clean_complex(nodes, level=0):
            for key, value in nodes.items():
                if isinstance(value, list):
                    for i, elem in enumerate(value):
                        if isinstance(elem, list):
                            for j, sub_elem in enumerate(elem):
                                if isinstance(sub_elem, list):
                                    for k, sub_sub_elem in enumerate(sub_elem):
                                        nodes[key][i][j][k] = clean_text(sub_sub_elem)
                                else:
                                    nodes[key][i][j] = clean_text(sub_elem)
                        else:
                            nodes[key][i] = clean_text(elem)
                elif isinstance(value, dict):
                    browse_clean_complex(value, level + 1)
            return nodes

        
        # Load the nodes
        nodes = data['text']
        browse_clean_complex(nodes, level=0)

    # Create a new directory named "cleaned" if it doesn't exist
    if not os.path.exists('cleaned'):
        os.makedirs('cleaned')

    # Define the cleaned filename with spaces replaced by underscores
    base_filename = os.path.basename(filename)
    cleaned_filename = os.path.join('cleaned', base_filename.replace(' ', '_').split('.')[0] + "_clean.json")
    
    
    # Save the cleaned file in the "cleaned" directory
    with open(cleaned_filename, "w") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

In [18]:
browse_and_clean('Babylon_Talmud_complete.json')

/tmp/ipykernel_156758/3242838570.py:34: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


In [19]:
def browse_and_show_unique_chars(filename):
    '''
    Show unique characters in the file, depending on the structure of the file (simple or complex, book or corpus)
    '''
    if identify_book_structure(filename) == 'book_simple':
        return browse_simple_book_structure_and_show_unique_chars(filename)
    elif identify_book_structure(filename) == 'book_complex':
        return browse_complex_book_structure_and_show_unique_chars(filename)
    elif identify_book_structure(filename) == 'corpus_simple':
        return browse_simple_corpus_structure_and_show_unique_chars(filename)
    else:
        return 'Unknown structure, cannot browse the file.'

In [43]:
unique_chars = browse_file_and_show_unique_chars('cleaned/Babylon_Talmud_complete_clean.json')
for c in unique_chars:
    print(f"{c} : {ord(c)}")

] : 93
. : 46
) : 41
׃ : 1475
ם : 1501
ך : 1498
פ : 1508
צ : 1510
· : 183
ט : 1496
נ : 1504
" : 34
ר : 1512
׀ : 1472
ש : 1513
ג : 1490
‘ : 8216
ל : 1500
/ : 47
- : 45
ו : 1493
: : 58
' : 39
ת : 1514
ק : 1511
, : 44
ץ : 1509
ס : 1505
ח : 1495
״ : 1524
( : 40
ע : 1506
מ : 1502
[ : 91
ה : 1492
־ : 1470
ן : 1503
׳ : 1523
װ : 1520
; : 59
י : 1497
ף : 1507
͏ : 847
| : 124
  : 32
ז : 1494
כ : 1499
ב : 1489
ד : 1491
א : 1488


In [44]:
def concatenate_verses(filename):
    '''
    Concatenate the verses of a book into a single text file.
    Handles both simple and complex structured files.
    '''
    with open(filename, "r") as f:
        data = json.load(f)

    concatenated_text = ""

    text_structure = identify_book_structure(filename)

    if text_structure == 'corpus_simple':
        for book in data:
            for chapter in book["text"]:
                for verse in chapter:
                    if isinstance(verse, str):
                        concatenated_text += verse
                    elif isinstance(verse, list):
                        for sub_verse in verse:
                            concatenated_text += sub_verse

    elif isinstance(data['text'], list):
        # Browse and clean verses directly in the data structure
        for chapter in data["text"]:
            for verse in chapter:
                if isinstance(verse, str):
                    concatenated_text += verse
                elif isinstance(verse, list):
                    for sub_verse in verse:
                        if isinstance(sub_verse, str):
                            concatenated_text += sub_verse

    elif isinstance(data['text'], dict):

        def browse_concat_complex(nodes):
            text = ""
            for value in nodes.values():
                if isinstance(value, list):
                    for elem in value:
                        if isinstance(elem, str):
                            text += elem
                        elif isinstance(elem, list):
                            for sub_elem in elem:
                                if isinstance(sub_elem, str):
                                    text += sub_elem
                                elif isinstance(sub_elem, list):
                                    for sub_sub_elem in sub_elem:
                                        if isinstance(sub_sub_elem, str):
                                            text += sub_sub_elem
                                        elif isinstance(sub_sub_elem, list):
                                            print("Nested verses detected and not handled!")
                elif isinstance(value, dict):
                    text += browse_concat_complex(value)
            return text

        # Load the nodes
        concatenated_text = browse_concat_complex(data['text'])
    
    else:
        print("Unknown structure, cannot concatenate the file.")

    # Create a new directory named "concatenated" if it doesn't exist
    concatenated_dir = 'concatenated'
    if not os.path.exists(concatenated_dir):
        os.makedirs(concatenated_dir)

    # Save the concatenated text to a file, in the "concatenated" directory
    concatenated_filename = os.path.join(concatenated_dir, os.path.splitext(os.path.basename(filename))[0] + "_concatenated.txt")
    with open(concatenated_filename, "w") as f:
        f.write(concatenated_text)


In [49]:
filename = 'cleaned/Jerusalem_Talmud_complete_clean.json'
concatenate_verses(filename)

## Pipeline

In [11]:
# For all the files in the directory, browse, clean and concatenate the verses
import os
path = "../../Corpuses_from_Sefaria/Bavli"

# Variable to track whether an error has occurred
error_occurred = False

# Clean the texts in the directory
for filename in os.listdir(path):
    if filename.endswith(".json"):
        try:
            browse_and_clean(os.path.join(path, filename))
        except Exception as e:
            error_occurred = True
            print(f"An error occurred while processing file {filename}: {str(e)}")

# create an index
for filename in os.listdir("cleaned"):
    if filename.endswith(".json"):
        try:
            create_index(os.path.join("cleaned", filename))
        except Exception as e:
            error_occurred = True
            print(f"An error occurred while creating index for file {filename}: {str(e)}")

# Concatenate the verses in the 'cleaned' directory
for filename in os.listdir("cleaned"):
    if filename.endswith(".json"):
        try:
            concatenate_verses(os.path.join("cleaned", filename))
        except Exception as e:
            error_occurred = True
            print(f"An error occurred while concatenating verses for file {filename}: {str(e)}")

# Afficher un message indiquant que le processus s'est terminé sans erreur
if not error_occurred:
    print("The process was completed without error.")

The process was completed without error.


In [12]:
# import os

# path = "../../Corpuses_from_Sefaria/Liturgy"

# # Variable to track whether an error has occurred
# error_occurred = False

# # Clean the texts in the directory
# for filename in os.listdir(path):
#     if filename.endswith(".json"):
#         browse_and_clean(os.path.join(path, filename))

# # create an index
# for filename in os.listdir("cleaned"):
#     if filename.endswith(".json"):
#         create_index(os.path.join("cleaned", filename))

# # Concatenate the verses in the 'cleaned' directory
# for filename in os.listdir("cleaned"):
#     if filename.endswith(".json"):
#         concatenate_verses(os.path.join("cleaned", filename))

# # Afficher un message indiquant que le processus s'est terminé sans erreur
# if not error_occurred:
#     print("The process was completed without error.")
